<a href="https://colab.research.google.com/github/Midmost/AI/blob/master/EDA_gu_0511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Korean District in Seoul budget EDA

## Dependencies

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install --upgrade pandas6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Basic information

In [28]:
import pandas as pd
import numpy as np
from pandas.io.formats.info import DataFrameInfo


# excel path
xlsx_file_path = "/content/All_district.xlsx" # TODO: 폴더를 OS로 읽고 한 번에 읽게

# pure excel to pure dataframe and information with a visual-table
df = pd.read_excel(xlsx_file_path)
DataFrameInfo


pandas.io.formats.info.DataFrameInfo

용량이 워낙 크다보니 부분 활용을 하려고 한다

In [29]:
# Select rows 0 to 20 (inclusive)
# df_subset = df.iloc[0:21]
# df_subset

df = df.iloc[0:2000]

## Pre-processing

#### pict01 

- [ ]  A규칙: `acct2`열에는 `2)재정계획심의위원회 참석 수당` 와 같은 값이 있다
- [ ]  B규칙: `acct2`열에는 `70,000원*12명*5회*80%` 와 같은 형식의 값이 있다
- [ ]  C규칙: A와 B를 만족하면서 이 둘에 해당하는 값이 concat되어 i행에 있어야한다.
- [ ]  if B규칙에 해당하는 값이 i+1행에 있는 지 확인
- [ ]  true 라면  i행에 있는 값과 concat한다
- [ ]  i+1 행에 있는 값은 지운다
- [ ]  A_1 규칙: `budget22`열에는 `3360` 와 같은 형식의 값이 있어야 한다
- [ ]  if A_1 규칙에 해당하는 행을 i+1행이라고 한다면 i행이 NULL인지 확인한다
- [ ]  true라면 i+1행의 값을 i행으로 replace한다

In [ ]:
# A: Find rows where 'acct2' column contains values like '2)재정계획심의위원회 참석 수당'
mask_A = df['bizdetail'].str.contains(r'\d+\).+')

# B: Find rows where 'acct2' column contains values like '70,000원*12명*5회*80%'
mask_B = df['bizdetail'].str.contains(r'\d+원\*\d+명\*\d+회\*\d+%')

But mask_B isn't fit with my intent, so I changed rule B

'*' 또는 '원' 또는 '명' 또는 '회' 또는 '%' 거나 숫자 이 중 하나라도 포함되는 경우는 정규식으로

##### 예시

In [20]:
import re

# Define the regular expression pattern
pattern = r'[*원명회%,]'

# Test some strings
print(bool(re.search(pattern, '70,000원*12명*5회*80%'))) # True
print(bool(re.search(pattern, 'abcdefg'))) # False

True
False


#### 규칙 B를 적용

그 전에 NaN 이 있는 지 확인하고 있다면 "exception"이란 이름으로 저장하자

In [34]:
# is there any NaN value in 'bizdetail'?

# Check if there are any NaN values in the 'bizdetail' column
has_nan = df['bizdetail'].isnull().any()

if has_nan:
    print("There are NaN values in the 'bizdetail' column.")
else:
    print("There are no NaN values in the 'bizdetail' column.")

# Count the number of NaN values in the 'bizdetail' column
num_nan = df['bizdetail'].isnull().sum()

print(f"There are {num_nan} NaN values in the 'bizdetail' column.")

# Replace NaN values in the 'bizdetail' column with the string "exception"
df['bizdetail'] = df['bizdetail'].fillna("exception")

There are no NaN values in the 'bizdetail' column.
There are 0 NaN values in the 'bizdetail' column.


Check the mask whether it is made properly or not. 

In [37]:
# Find rows where the 'acct2' column contains at least one of the characters '*', '원', '명', '회', '%' or a ','
mask = df['bizdetail'].str.contains(r'[*원명회%,]')

# Replace NaN values in the mask with False
mask = mask.fillna(False)

# Select the rows where the mask is True
df_subset = df[mask]
df_subset

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
6,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"3,700,000원*3회",NaN,"3,700,000원*3회",11100.0,NaN,NaN,7
7,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"1)의정활동 홍보 광고비835,000원*70회",NaN,"835,000원*70회",58450.0,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"1)감리비1,244,000,000원*0.4",NaN,"1,244,000,000원*0.4",497600.0,NaN,NaN,1992
1993,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"1)시설부대비69,600,000원*1식",NaN,"69,600,000원*1식",69600.0,NaN,NaN,1994
1995,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"69,000,000원*1.4",NaN,"69,000,000원*1.4",96600.0,NaN,NaN,1996
1997,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"60,000,000원*1.4",NaN,"60,000,000원*1.4",84000.0,NaN,NaN,1998


23행 같은 경우는 detailamt와 budget22가 한 칸 위로 옮겨져서는 안 된다. 

따라서 규칙을 추가해줘야 한다. 

만약 i행이 bizdetail에서 1)로 시작하는 경우이면서 budget22가 값이 있고 i+1행이 2)로 시작한다면 i행의 detailamt와 budget22 값은 한 칸 위로 옮겨져서는 안 된다. 

In [38]:
# Create a boolean mask where True indicates that the cell value starts with '1)'
mask_1 = df['bizdetail'].str.startswith('1)')

# Create a boolean mask where True indicates that the cell value starts with '2)'
mask_2 = df['bizdetail'].str.startswith('2)')

# Find the indices of the rows where the mask is False
indices = df[~mask].index

# Iterate over the indices
for i in indices:
    # Check if the current row satisfies the conditions
    if not (mask_1[i] and pd.notnull(df.at[i, 'budget22']) and mask_2[i + 1]):
        # Shift the values of the "detailamt" and "budget22" columns one cell up for this row
        df.loc[i, ["detailamt", "budget22"]] = df.loc[i + 1, ["detailamt", "budget22"]].values

In [39]:
df

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
0,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,NaN,01 사무관리비,01 사무관리비,NaN,69500.0,193770.0,"△38,870",1
1,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정 홍보물 제작,NaN,NaN,69500.0,NaN,NaN,2
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"69,000,000원*1.4",NaN,"69,000,000원*1.4",96600.0,NaN,NaN,1996
1996,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)각종 본인증 용역 및 수수료,NaN,"60,000,000원*1.4",84000.0,NaN,NaN,1997
1997,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"60,000,000원*1.4",NaN,"60,000,000원*1.4",84000.0,NaN,NaN,1998
1998,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,406 기타자본이전,본예산,01 기타자본이전,01 기타자본이전,"450,000,000원*1식",450000.0,0.0,450000,1999


In [17]:
# # Find the indices of the rows where the mask is False
# indices = df[~mask].index

# # Shift the values of the "detailamt" and "budget22" columns one cell up for these rows
# df.loc[indices, ["detailamt", "budget22"]] = df.loc[indices + 1, ["detailamt", "budget22"]].values
# df

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
0,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,NaN,01 사무관리비,01 사무관리비,NaN,69500.0,193770.0,"△38,870",1
1,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정 홍보물 제작,NaN,"11,000원*1,500부*1회",16500.0,NaN,NaN,2
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
5,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정활동 홍보 대형 현수막,NaN,"3,700,000원*3회",11100.0,NaN,NaN,6
6,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"3,700,000원*3회",NaN,"3,700,000원*3회",11100.0,NaN,NaN,7
7,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"1)의정활동 홍보 광고비835,000원*70회",NaN,"835,000원*70회",58450.0,NaN,NaN,8
8,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)사진현상 인화 및 용품 구매,NaN,"300,000원*12월",3600.0,NaN,NaN,9
9,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"300,000원*12월",NaN,"300,000원*12월",3600.0,NaN,NaN,10


In [11]:


# # Create a boolean mask where both conditions are True
# mask_C = mask_A & mask_B

# # Find the indices of the rows where the mask is True
# indices = df[mask_C].index

indices = df[mask_B].index


# Iterate over the indices
for i in indices:
    # Check if the next row also satisfies condition B
    if mask_B[i + 1]:
        # Concatenate the values of the current and next rows
        df.at[i, 'bizdetail'] = df.at[i, 'bizdetail'] + df.at[i + 1, 'bizdetail']
        # Set the value of the next row to an empty string
        df.at[i + 1, 'bizdetail'] = ''

# Find rows where 'budget22' column contains values like '3360'
mask_A1 = df['budget22'].astype(str).str.match(r'\d+')

# Find the indices of the rows where the mask is True
indices = df[mask_A1].index

# Iterate over the indices
for i in indices:
    # Check if the value of the current row is NULL
    if pd.isnull(df.at[i, 'budget22']):
        # Replace the value of the current row with the value of the next row
        df.at[i, 'budget22'] = df.at[i + 1, 'budget22']

In [12]:
df

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
0,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,NaN,01 사무관리비,01 사무관리비,NaN,154900.0,193770.0,"△38,870",1
1,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정 홍보물 제작,NaN,NaN,69500.0,NaN,NaN,2
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
5,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정활동 홍보 대형 현수막,NaN,NaN,NaN,NaN,NaN,6
6,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"3,700,000원*3회",NaN,"3,700,000원*3회",11100.0,NaN,NaN,7
7,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"1)의정활동 홍보 광고비835,000원*70회",NaN,"835,000원*70회",58450.0,NaN,NaN,8
8,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)사진현상 인화 및 용품 구매,NaN,NaN,NaN,NaN,NaN,9
9,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"300,000원*12월",NaN,"300,000원*12월",3600.0,NaN,NaN,10


### Put the merged column(bizdetail)

##### Make a new empty-column for bizdetail 

In [20]:
# Find the index of the '5' column
index = df.columns.get_loc(5)

# Insert a new column after it
df.insert(index + 1, 'new_column', '')

In [21]:
# df = df.drop(df.columns[9], axis=1)
# df.head(30)

##### Add headers *check needed

In [22]:
header_all = "gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff"
header = header_all.split(",")
df.columns = header

##### set the columns values

In [23]:
columns_to_fill ="gov,dept,policy,danwi,task,acct1,acct2,bizdetail".split(",") # '부서', '정책', '단위', '세부사업', '편성목', '편성운영비'
# columns_to_fill = header[:5] 이 방법으로도 가능

for col in columns_to_fill:
    df[col] = df[col].fillna(method='ffill')


fill df['bizdetail']

In [24]:
df['bizdetailadd'] = df['bizdetailadd'].astype(str).replace('nan', '')
df['detailamt'] = df['detailamt'].astype(str).replace('nan', '')
df['bizdetail'] = df['bizdetailadd'] + df['detailamt']

if df['bizdetailadd'] startwith('1)') then delete the cell's content

In [25]:
# Create a boolean mask where True indicates that the cell value starts with '1)'
mask = df['bizdetailadd'].str.startswith('1)')

# Set the values of the selected cells to an empty string
df.loc[mask, 'bizdetailadd'] = ''

In [26]:
# 1. DataFrame after dropping empty rows:
df = df[df['bizdetail'] != '']

In [27]:
df

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff
6,강북구,의회사무국,지방의회 운영 지원,의정활동 지원,의정공통업무 및 의장단 활동 지원,205 의회비,01 의정활동비,"1)의정활동비1,100,000원*14명*12회",,"1,100,000원*14명*12회",184800,NaN,NaN
8,강북구,의회사무국,지방의회 운영 지원,의정활동 지원,의정공통업무 및 의장단 활동 지원,205 의회비,02 월정수당,"1)월정수당2,769,080원*14명*12월",,"2,769,080원*14명*12월",465206,NaN,NaN
10,강북구,의회사무국,지방의회 운영 지원,의정활동 지원,의정공통업무 및 의장단 활동 지원,205 의회비,05 의정운영공통경비,1)의정운영 공통 업무추진,,,NaN,NaN,NaN
11,강북구,의회사무국,지방의회 운영 지원,의정활동 지원,의정공통업무 및 의장단 활동 지원,205 의회비,05 의정운영공통경비,"7,467,500원*14명",,"7,467,500원*14명",104545,NaN,NaN
13,강북구,의회사무국,지방의회 운영 지원,의정활동 지원,의정공통업무 및 의장단 활동 지원,205 의회비,06 의회운영업무추진비,1)의장단 활동비,,,116800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20453,강북구,주차관리과,행정운영경비,기본경비(주차장특별회계),기본경비(주차관리과),202 여비,01 국내여비,1)현안업무추진 여비(주정차 특별단속),,,NaN,NaN,NaN
20454,강북구,주차관리과,행정운영경비,기본경비(주차장특별회계),기본경비(주차관리과),202 여비,01 국내여비,"20,000원*9명*3일*12월",,"20,000원*9명*3일*12월",6480,NaN,NaN
20457,강북구,주차관리과,행정운영경비,기본경비(주차장특별회계),기본경비(주차관리과),405 자산취득비,01 자산및물품취득비,"1)컬러프린터 복합기6,430,000원",,"6,430,000원",6430,NaN,NaN
20464,강북구,주차관리과,재무활동(주차관리과),보전지출(주차장특별회계),통합재정안정화기금 예탁,704 예탁금,01 예탁금,1)통합재정안정화기금 예탁금,,,NaN,NaN,NaN


## Export

In [28]:
df = df.reset_index(drop=True) # to_excel에서 index를 버릴 수 있지만 혹시 모르니 넣어둠

# Export the DataFrame to an Excel file
df.to_excel('gangbuk_gu.xlsx', index=False)